In [ ]:
%load_ext watermark
%watermark

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
from sklearn import datasets

In [ ]:
from IPython.display import Image

Image("../../media/breast_cancer.jpeg")

In [ ]:
cancer_datos = datasets.load_breast_cancer()

cancer_df = pd.DataFrame(cancer_datos["data"],
                           columns=cancer_datos["feature_names"]
                          )

cancer_df["objetivo"] = cancer_datos.target
cancer_df["objetivo"] = cancer_df["objetivo"].replace({0:1, 1:0})

In [ ]:
cancer_df["objetivo"].value_counts(True)

In [ ]:
cancer_df.head()

En primer lugar creamos un modelo simple de Regresión Logística.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
X = cancer_df[cancer_datos.feature_names]
y = cancer_df["objetivo"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
modelo = LogisticRegression()

modelo.fit(X_train, y_train)

predicciones = modelo.predict(X_test)
clases_reales = y_test
predicciones_probabilidades = modelo.predict_proba(X_test)

In [ ]:
def tupla_clase_prediccion(y_real, y_pred):
    return list(zip(y_real, y_pred))

tupla_clase_prediccion(clases_reales, predicciones)[:10]

**Conceptos de Clasificación binaria**

En clasificación binaria, tenemos el concepto de casos negativos (clase 0, en el caso del dataset de cancer de mama serian los casos donde el cancer es benigno) y casos positivos (clase 1, en el caso del dataset de cancer de mama serían los casos donde el cancer es maligno). Positivo y negativo no significa que el resultado sea bueno o malo, simplemente que la detección de un cancer maligno se active o no.

- Casos positivos: Casos donde la clase es 1 (cánceres malignos)
- Casos negativos: Casos donde la clase es 0 (cánceres benignos)

Esto nos lleva a computar 4 tipos de observaciones, explicados como ejemplos del dataset del cancer de mama.

- Verdaderos Positivos(True positives), serían las imágenes con un cancer maligno que se detectan como cancer maligno.
- Falsos Positivos (False positives), serían los cánceres benignos que se detectan como un cancer maligno.
- Verdaderos Negativos(True Negatives), serían los canceres benignos que se clasifican como cánceres benignos.
- Falsos Negativos(False Negatives), serían los canceres malignos que se clasifican como cánceres benignos.

In [ ]:
Image("../../media/Precisionrecall.svg.png")

In [ ]:
def VP(clases_reales, predicciones):
    par_clase_prediccion = tupla_clase_prediccion(clases_reales, predicciones)
    return len([obs for obs in par_clase_prediccion if obs[0]==1 and obs[1]==1])

def VN(clases_reales, predicciones):
    par_clase_prediccion = tupla_clase_prediccion(clases_reales, predicciones)
    return len([obs for obs in par_clase_prediccion if obs[0]==0 and obs[1]==0])
    
def FP(clases_reales, predicciones):
    par_clase_prediccion = tupla_clase_prediccion(clases_reales, predicciones)
    return len([obs for obs in par_clase_prediccion if obs[0]==0 and obs[1]==1])

def FN(clases_reales, predicciones):
    par_clase_prediccion = tupla_clase_prediccion(clases_reales, predicciones)
    return len([obs for obs in par_clase_prediccion if obs[0]==1 and obs[1]==0])


print("""
Verdaderos Positivos: {}
Verdaderos Negativos: {}
Falsos Positivos: {}
Falsos Negativos: {}
""".format(
    VP(clases_reales, predicciones),
    VN(clases_reales, predicciones),
    FP(clases_reales, predicciones),
    FN(clases_reales, predicciones)    
))    

### Ratios de clasificación

**Exactitud (Accuracy)**

La exactitud es una medida general de como se comporta el modelo, mide simplemente el porcentaje de casos que se han clasificado correctamente.

$$Exactitud=\frac{Número~de~observaciones~correctamente~clasificadas}{Número~de~observaciones~totales}= \frac{VP+VN}{VP+VN+FP+FN}$$

In [ ]:
def exactitud(clases_reales, predicciones):
    vp = VP(clases_reales, predicciones)
    vn = VN(clases_reales, predicciones)
    return (vp+vn) / len(clases_reales)

exactitud(clases_reales, predicciones)

In [ ]:
from sklearn import metrics
metrics.accuracy_score(clases_reales, predicciones)

**Precisión (Precission)**

La precisión indica la habilidad del modelo para clasificar como positivos los casos que son positivos.

$$Precisión=\frac{Número~de~observaciones~positivas~correctamente~clasificadas}{Número~de~observaciones~clasificadas~como~positivas}= \frac{VP}{VP+FP}$$

In [ ]:
def precision(clases_reales, predicciones):
    vp = VP(clases_reales, predicciones)
    fp = FP(clases_reales, predicciones)
    return vp / (vp+fp)

precision(clases_reales, predicciones)

In [ ]:
metrics.average_precision_score(clases_reales, predicciones)

**Exhaustividad o sensibilidad(Recall o True Positive Rate)**

La sensibilidad nos da una medida de la habilidad del modelo para encontrar todos los casos positivos. La sensibilidad se mide en función de una clase.

$$Sensibilidad=\frac{Número~de~observaciones~positivas~clasificadas~como~positivas}{Número~de~observaciones~positivas~totales}= \frac{VP}{VP+FN}$$

In [ ]:
def sensibilidad(clases_reales, predicciones):
    vp = VP(clases_reales, predicciones)
    fn = FN(clases_reales, predicciones)
    return vp / (vp+fn)

sensibilidad(clases_reales, predicciones)

In [ ]:
metrics.recall_score(clases_reales, predicciones)

** Matriz de confusion **

La matriz de confusión es una forma muy sencilla de comparar como ha clasificado cada observación el modelo.

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(clases_reales, predicciones)

**Puntuación F1 (F1 score)**

La puntuación F1 es una media ponderada entre la sensibilidad (que intenta obtener cuantos mas verdaderos positivos independientemente de los falsos positivos) y la precisión (que intenta obtener solo verdaderos positivos que sean casos claros para limitar los falsos positivos).

La puntuación F1 se define como la media harmónica de la precisión y la sensibilidad:

$$F1=2*\frac{1}{\frac{1}{precisión}+\frac{1}{sensibilidad}}=2*\frac{precisión*sensibilidad}{precisión+sensibilidad}$$

In [ ]:
def puntuacion_f1(clases_reales, predicciones):
    precision_preds = precision(clases_reales, predicciones)
    sensibilidad_preds = sensibilidad(clases_reales, predicciones)
    return 2*(precision_preds*sensibilidad_preds)/(precision_preds+sensibilidad_preds)

puntuacion_f1(clases_reales, predicciones)

In [ ]:
metrics.f1_score(clases_reales, predicciones)

** Ratio de Falsos Positivos (Ratio de Falsa Alarma o FPR) **

El ratio de falsos positivos nos da una medida de las probabilidades de nuestro modelo de asignar una clase positiva a un caso negativo.

Se define como:

$$FPR=\frac{Número~de~observaciones~negativas~clasificadas~como~positivas}{Número~de~observaciones~totales}= \frac{FP}{FP+TN}$$

In [ ]:
def fpr(clases_reales, predicciones):
    return (FP(clases_reales, predicciones) / (
             FP(clases_reales, predicciones) + VN(clases_reales, predicciones)
             )
           )
fpr(clases_reales, predicciones)

**¿Cómo clasifica un modelo?**

Un modelo como la regresión lineal funciona prediciendo distancias a una "linea de decision" que se convierten en probabilidades para cada clase. Pero a la hora de la verdad al modelo le suele interesar sólo saber que clase predice el modelo. En general esto se hace decidiendo un umbral *(threshold)* y clasificando los casos con menor probabilidad como clase negativa y  mayor probabilidad como clase positiva.

In [ ]:
Image("../../media/threshold.png")

In [ ]:
df = pd.DataFrame({"clase_real":clases_reales,
                   "clase_pred": predicciones,
                   "probabilidades_0":modelo.predict_proba(X_test)[:,0],
                    "probabilidades_1":modelo.predict_proba(X_test)[:,1],
                  })
df["sum_probas"] = df.probabilidades_0 + df.probabilidades_1

In [ ]:
df.sample(10)

Como el modelo no tiene ningún motivo para elegir un umbral determinado (sólo sabe probabilidades) elige 0.5 por defecto.

In [ ]:
df.query("probabilidades_1>0.5 & clase_pred==0")

In [ ]:
df.query("probabilidades_0>0.5 & clase_pred==1")

**Curva Precisión (Precission-Recall Curve)**

In [ ]:
def probabilidades_a_clases(predicciones_probabilidades, umbral=0.5):
    predicciones = np.zeros([len(predicciones_probabilidades), ])
    predicciones[predicciones_probabilidades[:,1]>=umbral] = 1
    return predicciones

In [ ]:
predicciones_probabilidades[:10]

In [ ]:
probabilidades_a_clases(predicciones_probabilidades, umbral=0.90)[:10]

In [ ]:
from ipywidgets import widgets, fixed, interact
@interact(umbral=widgets.FloatSlider(min=0.01, max=0.99, step=0.01, value=0.01))
def evaluar_umbral(umbral):
    predicciones_en_umbral = probabilidades_a_clases(predicciones_probabilidades, umbral)
    sensibilidad_umbral = metrics.recall_score(clases_reales, predicciones_en_umbral)
    fpr_umbral = fpr(clases_reales, predicciones_en_umbral)
    precision_umbral = precision(clases_reales, predicciones_en_umbral) 
    print( """
    Precision: {:.3f}
    Sensibilidad:{:.3f}
    Ratio de Alarma: {:.3f}
    """.format(
        precision_umbral,
        sensibilidad_umbral,
        fpr_umbral
    ))

In [ ]:
def evaluar_umbral(umbral):
    predicciones_en_umbral = probabilidades_a_clases(predicciones_probabilidades, umbral)
    precision_umbral = precision(clases_reales, predicciones_en_umbral)
    sensibilidad_umbral = metrics.recall_score(clases_reales, predicciones_en_umbral)
    fpr_umbral = fpr(clases_reales, predicciones_en_umbral)
    return precision_umbral, sensibilidad_umbral, fpr_umbral


rango_umbral = np.linspace(0., 1., 1000)
sensibilidad_umbrales = []
precision_umbrales = []
fpr_umbrales = []

for umbral in rango_umbral:
    precision_umbral, sensibilidad_umbral, fpr_umbral = evaluar_umbral(umbral)
    precision_umbrales.append(precision_umbral)
    sensibilidad_umbrales.append(sensibilidad_umbral)
    fpr_umbrales.append(fpr_umbral)

In [ ]:
plt.plot(sensibilidad_umbrales, precision_umbrales);
plt.ylabel("Precision")
plt.xlabel("Ratio de Verdaderos positivos (sensibilidad)")
plt.title("Curva Precision-Recall");

In [ ]:
def grafica_precision_recall(clases_reales, predicciones_probabilidades):
    precision_, recall_, _ = metrics.precision_recall_curve(
        clases_reales, predicciones_probabilidades[:,1])

    plt.step(recall_, precision_, color='b', alpha=0.2,
         where='post')
    plt.fill_between(recall_, precision_, step='post', alpha=0.2,
                 color='b')

    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.05])
    plt.title('Curva Precision-Recall');
    plt.show()


grafica_precision_recall(clases_reales, predicciones_probabilidades)

**Area bajo la curva (Area Under the Curve, ROC-AUC)**

In [ ]:
plt.plot(fpr_umbrales, sensibilidad_umbrales);
plt.xlabel("Ratio de Falsos positivos (FPR)")
plt.ylabel("Ratio de Verdaderos positivos (sensibilidad)")
plt.title("Curva ROC");

In [ ]:
metrics.roc_auc_score(clases_reales, predicciones)

In [ ]:
def grafica_curva_auc(clases_reales, predicciones, predicciones_probabilidades):
    fpr, tpr, _ = metrics.roc_curve(clases_reales, predicciones_probabilidades[:,1])
    roc_auc = metrics.roc_auc_score(clases_reales, predicciones)
    plt.figure()

    plt.plot(fpr, tpr, color='darkorange',
         lw=2, label='Curva ROC (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', label="estimador aleatorio")
    
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('FPR')
    plt.ylabel('TPR (recall)')
    plt.title('Curva ROC')
    plt.legend(loc="lower right")
    plt.show();

grafica_curva_auc(clases_reales, predicciones, predicciones_probabilidades)

In [ ]:
def evaluar_modelo(clases_reales, predicciones, probabilidades):
    exactitud = metrics.accuracy_score(clases_reales, predicciones)
    precision = metrics.average_precision_score(clases_reales, predicciones)
    sensibilidad = metrics.recall_score(clases_reales, predicciones)
    roc_auc = metrics.roc_auc_score(clases_reales, predicciones)
    f1 = metrics.f1_score(clases_reales, predicciones)
    print("""
    Exactitud: {:.3f}
    Precisión: {:.3f}
    Sensibilidad: {:.3f}
    Area bajo curva (AUC): {:.3f}
    Puntuación F1: {:.3f}
    """.format(
        exactitud, 
        precision,
        sensibilidad,
        roc_auc,
        f1
    ))
    
evaluar_modelo(clases_reales, predicciones, predicciones_probabilidades)

<hr>
<hr>
<hr>

In [ ]:
cancer_df.objetivo.value_counts(True)

# Evaluación en Dataset imbalanceado

Ahora vamos a usar un dataset con clases no balanceadas

https://www.kaggle.com/c/GiveMeSomeCredit/data


>Improve on the state of the art in credit scoring by predicting the probability that somebody will experience financial distress in the next two years. 

In [ ]:
creditos_df = pd.read_csv("data/datos_creditos.csv")

In [ ]:
creditos_df.head()

In [ ]:
variable_objetivo = "impago_en_2_anos"

In [ ]:
X = creditos_df.drop(variable_objetivo, axis=1)
y = creditos_df[variable_objetivo]

X_train_credito, X_test_credito, y_train_credito, y_test_credito = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [ ]:
y.value_counts(normalize=True)

In [ ]:
modelo = LogisticRegression()

modelo.fit(X_train_credito, y_train_credito)

predicciones_creditos = modelo.predict(X_test_credito)
clases_reales_creditos = y_test_credito
predicciones_probabilidades_creditos = modelo.predict_proba(X_test_credito)

In [ ]:
evaluar_modelo(clases_reales_creditos, predicciones_creditos, predicciones_probabilidades_creditos)

In [ ]:
len(creditos_df[creditos_df[variable_objetivo]==1])

In [ ]:
len([pred for pred in predicciones if pred==1])

In [ ]:
grafica_precision_recall(clases_reales_creditos, predicciones_probabilidades_creditos)

In [ ]:
grafica_curva_auc(clases_reales_creditos, predicciones_creditos, predicciones_probabilidades_creditos)



## Como decidir un Umbral de decision

In [209]:
import pandas as pd
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

cancer_datos = datasets.load_breast_cancer()

cancer_df = pd.DataFrame(cancer_datos["data"],
                           columns=cancer_datos["feature_names"]
                          )

cancer_df["objetivo"] = cancer_datos.target
cancer_df["objetivo"] = cancer_df["objetivo"].replace({0:1, 1:0})

In [210]:
X = cancer_df[cancer_datos.feature_names]
y = cancer_df["objetivo"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [211]:
modelo = LogisticRegression()

modelo.fit(X_train, y_train)

predicciones = modelo.predict(X_test)
clases_reales = y_test
predicciones_probabilidades = modelo.predict_proba(X_test)

In [212]:
probas = modelo.predict_proba(X_test)[:5]
probas

array([[  8.15760088e-01,   1.84239912e-01],
       [  4.18456803e-09,   9.99999996e-01],
       [  2.28673475e-03,   9.97713265e-01],
       [  9.96495038e-01,   3.50496189e-03],
       [  9.99046504e-01,   9.53496104e-04]])

In [213]:
umbral_decision = 0.5

probas[:,1]>=umbral_decision

array([False,  True,  True, False, False], dtype=bool)

In [214]:
umbral_decision = 0.1

probas[:,1]>=umbral_decision

array([ True,  True,  True, False, False], dtype=bool)

In [215]:
def softmax(coste_fp, coste_fn):
    return np.exp(coste_fp) / (np.exp(coste_fn)+np.exp(coste_fp))

coste_fn = 1
coste_fp = 2
softmax(coste_fp, coste_fn)

0.7310585786300049

In [216]:
from ipywidgets import widgets, interact

@interact
def calculo_umbral(
    coste_fp=widgets.FloatSlider(min=1, max=10, step=0.1, value=1),
    coste_fn=widgets.FloatSlider(min=1, max=10, step=0.1, value=1),
):
    return softmax(coste_fp, coste_fn)

In [229]:
coste_fn = 10
coste_fp = 1
umbral_decision = calculo_umbral(coste_fp, coste_fn)
print(umbral_decision)
decisiones = probabilidades_a_clases(probas, umbral_decision)
decisiones

0.000123394575986


array([ 1.,  1.,  1.,  1.,  1.])

In [218]:
class BusinessLogisticRegression(LogisticRegression):
        
    def decision_de_negocio(self, X, coste_fp=1, coste_fn=1, *args, **kwargs):
        probs = self.predict_proba(X)
        umbral_decision = calculo_umbral(coste_fp, coste_fn)
        print("Umbral de decision: {}".format(umbral_decision))
        decisiones = probabilidades_a_clases(probs, umbral_decision)
        return decisiones
        
modelo_negocio = BusinessLogisticRegression()

modelo_negocio.fit(X_train, y_train)        

BusinessLogisticRegression(C=1.0, class_weight=None, dual=False,
              fit_intercept=True, intercept_scaling=1, max_iter=100,
              multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
              solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [219]:
modelo_negocio.predict(X_test[:5])

array([0, 1, 1, 0, 0])

In [220]:
modelo_negocio.predict_proba(X_test[:5])

array([[  8.15760088e-01,   1.84239912e-01],
       [  4.18456803e-09,   9.99999996e-01],
       [  2.28673475e-03,   9.97713265e-01],
       [  9.96495038e-01,   3.50496189e-03],
       [  9.99046504e-01,   9.53496104e-04]])

In [223]:
modelo_negocio.decision_de_negocio(X_test[:5], 1, 1)

Umbral de decision: 0.0001233945759862317


array([ 1.,  1.,  1.,  1.,  1.])

In [224]:
@interact(
    coste_fp=widgets.FloatSlider(min=1.,max=10.,step=.1,value=1.),
    coste_fn=widgets.FloatSlider(min=1.,max=10.,step=.1,value=1.)
)
def decision_negocio(coste_fp, coste_fn):
    predicciones = modelo_negocio.decision_de_negocio(X_test, coste_fp, coste_fn)
    print(confusion_matrix(clases_reales, predicciones))